In [17]:
import pandas as pd
import numpy as np
import re
import pandas as pd

import matcher  
import spacy


In [18]:
# Cargar los datos del archivo CSV
df = matcher.get_df('colores.csv')

In [21]:
nlp = spacy.load("en_core_web_sm")

In [39]:
nlp = spacy.load("es_core_news_sm")

# Crear un DataFrame de ejemplo
data = {
    "texto": ["Esto es una prueba de lematización.", "Los gatos están durmiendo."],
}
df = pd.DataFrame(data)

# Definir una función para lematizar el texto en la columna
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

# Aplicar la función a la columna de texto
df["lemmatized_text"] = df["texto"].apply(lemmatize_text)

# Mostrar el DataFrame resultante
print(df)

                                 texto                        lemmatized_text
0  Esto es una prueba de lematización.  este ser uno prueba de lematización .
1           Los gatos están durmiendo.                 el gato estar dormir .


In [32]:
for token in df['color']:
    token = nlp(token)
    # if token.is_punct or token.is_stop or token.is_space:
        # continue
    print(token)
    print(token.lemma_)

Rojo


AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'lemma_'

In [36]:

# Process a text
doc = nlp("I am running in the park")

# Access lemmatized form of each token in the document
for token in doc:
    print(token.text, token.lemma_)

I I
am be
running run
in in
the the
park park


In [6]:
train_data

[[('Rojo', 'COLOR'),
  ('Azul', 'COLOR'),
  ('Amarillo', 'COLOR'),
  ('Naranja', 'COLOR'),
  ('Verde', 'COLOR'),
  ('Violeta', 'COLOR'),
  ('Anaranjado', 'COLOR'),
  ('Morado', 'COLOR'),
  ('Verde lima', 'COLOR'),
  ('Blanco', 'COLOR'),
  ('Negro', 'COLOR'),
  ('Gris', 'COLOR'),
  ('Red', 'COLOR'),
  ('Blue', 'COLOR'),
  ('Yellow', 'COLOR'),
  ('Orange', 'COLOR'),
  ('Green', 'COLOR'),
  ('Violet', 'COLOR'),
  ('Vermilion', 'COLOR'),
  ('Chartreuse', 'COLOR'),
  ('Magenta', 'COLOR'),
  ('White', 'COLOR'),
  ('Black', 'COLOR'),
  ('Gray', 'COLOR'),
  ('Rouge', 'COLOR'),
  ('Bleu', 'COLOR'),
  ('Jaune', 'COLOR'),
  ('Orange', 'COLOR'),
  ('Vert', 'COLOR'),
  ('Violet', 'COLOR'),
  ('Rougeâtre', 'COLOR'),
  ('Vert olive', 'COLOR'),
  ('Jeanne citron', 'COLOR'),
  ('Blanc', 'COLOR'),
  ('Noir', 'COLOR'),
  ('Gris', 'COLOR'),
  ('Красный (Krasniy)', 'COLOR'),
  ('Синий (Siniy)', 'COLOR'),
  ('Желтый (Zheltiy)', 'COLOR'),
  ('Оранжевый (Oranzhevyy)', 'COLOR'),
  ('Зеленый (Zeleniy)', 'COLOR'

In [13]:
matcher.detect_language_columns('files/farmatodo.json')

{'title': 'es', 'price': 'de', 'url': 'Unknown', 'input': 'es'}

In [7]:
lemmatized_colors = set(lemmatized_colors)


In [8]:
# Entrenar el etiquetador de unigramas
train_data = [[(color, 'COLOR') for color in lemmatized_colors]]
unigram_tagger = UnigramTagger(train_data)

In [14]:
unigram_tagger

<UnigramTagger: size=57>

In [9]:
def es_color(palabra):
    tagged_word = unigram_tagger.tag([palabra])
    if tagged_word[0][1] == 'COLOR':
        return True
    else:
        return False

In [11]:
es_color('rojizo')

False

In [22]:
# Función para verificar si una lista de palabras contiene colores
def contiene_colores(lista_palabras):
    return [palabra for palabra in lista_palabras if es_color(palabra)]

# Ejemplo de uso
lista_palabras = ['Rojo', 'Perro', 'Azul', 'Gato']
print(contiene_colores(lista_palabras)) # Output: ['Rojo', 'Azul']

['Rojo', 'Azul']


In [ ]:
# Convertir las etiquetas de categoría a números
y = y.map({cat: i for i, cat in enumerate(categorias)})

In [3]:
df = matcher.get_df('files/origin_Madison.csv')

In [8]:
df.apply(pd.unique(df))

C:\Users\Caritoshi\AppData\Local\Temp\ipykernel_13456\1888121351.py:1: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  df.apply(pd.unique(df))


ValueError: could not broadcast input array from shape (120,20) into shape (120,)